In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine,correlation


In [2]:
data = pd.read_csv('./ml-100k/u.data')
data.head()

,196 242 3 881250949
0,186\t302\t3\t891717742
1,22\t377\t1\t878887116
2,244\t51\t2\t880606923
3,166\t346\t1\t886397596
4,298\t474\t4\t884182806


In [3]:
data = pd.read_csv('./ml-100k/u.data',sep = '\t', names = ['user_id','item_id','rating','timstamp'])

In [4]:
data.head()

,user_id,item_id,rating,timstamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
user_id     100000 non-null int64
item_id     100000 non-null int64
rating      100000 non-null int64
timstamp    100000 non-null int64
dtypes: int64(4)
memory usage: 3.1 MB


In [6]:
data.describe()

,user_id,item_id,rating,timstamp
count,100000.00000,100000.000000,100000.000000,1.000000e+05
mean,462.48475,425.530130,3.529860,8.835289e+08
std,266.61442,330.798356,1.125674,5.343856e+06
min,1.00000,1.000000,1.000000,8.747247e+08
25%,254.00000,175.000000,3.000000,8.794487e+08
50%,447.00000,322.000000,4.000000,8.828269e+08
75%,682.00000,631.000000,4.000000,8.882600e+08
max,943.00000,1682.000000,5.000000,8.932866e+08


In [7]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data,test_size = 0.3) 
# 训练集与测试集70:30比例拆分

In [8]:
train_data.describe()

,user_id,item_id,rating,timstamp
count,70000.000000,70000.000000,70000.000000,7.000000e+04
mean,461.943386,424.653286,3.526643,8.835395e+08
std,266.542385,330.327717,1.128182,5.340809e+06
min,1.000000,1.000000,1.000000,8.747247e+08
25%,254.000000,175.000000,3.000000,8.794518e+08
50%,446.000000,321.000000,4.000000,8.828293e+08
75%,682.000000,631.000000,4.000000,8.882600e+08
max,943.000000,1682.000000,5.000000,8.932866e+08


In [9]:
# 此时的data就为用户评分表，即复现中的movie_rating
# 创建用户-电影评分表
rating_matrix = train_data.pivot_table(index = ['item_id'],columns = ['user_id'],
                                values = 'rating').reset_index(drop = True)
rating_matrix.fillna(0, inplace = True)
user_item = rating_matrix
user_item.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
0,5.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,4.0,...,2.0,3.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0
1,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
4,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
user_item.shape

(1640, 943)

In [11]:
# 训练集构造见物品相似矩阵
movie_similarity = 1 - pairwise_distances(rating_matrix.values, metric = "cosine")
np.fill_diagonal(movie_similarity,0)
rating_matrix = pd.DataFrame(movie_similarity)
rating_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,1630,1631,1632,1633,1634,1635,1636,1637,1638,1639
0,0.000000,0.304061,0.238346,0.331984,0.207967,0.070547,0.419144,0.340349,0.332658,0.167796,...,0.0,0.055685,0.041763,0.0,0.0,0.041763,0.0,0.0,0.0,0.055685
1,0.304061,0.000000,0.175394,0.367031,0.276769,0.063557,0.238337,0.226796,0.208796,0.114723,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.089883
2,0.238346,0.175394,0.000000,0.222119,0.109961,0.033078,0.259382,0.108569,0.157483,0.112145,...,0.0,0.000000,0.000000,0.0,0.0,0.040096,0.0,0.0,0.0,0.120289
3,0.331984,0.367031,0.222119,0.000000,0.307155,0.079616,0.350834,0.388545,0.274337,0.170954,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
4,0.207967,0.276769,0.109961,0.307155,0.000000,0.048905,0.265924,0.164031,0.190640,0.042052,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.108253


In [12]:
rating_matrix.shape

(1640, 1640)

In [13]:
movie_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']
movie = pd.read_csv('./ml-100k/u.item', sep = '|', names = movie_cols, encoding = 'latin-1',usecols = ["movie_id","title","release_date","video_release_date","imdb_url"])
movie.drop(movie.columns[[3,4]], axis = 1, inplace =  True)
movie.head()

,movie_id,title,release_date
0,1,Toy Story (1995),01-Jan-1995
1,2,GoldenEye (1995),01-Jan-1995
2,3,Four Rooms (1995),01-Jan-1995
3,4,Get Shorty (1995),01-Jan-1995
4,5,Copycat (1995),01-Jan-1995


In [14]:
# 训练集推荐TOP_N
user_inp = "Four Rooms (1995)" 
inp = movie[movie['title'] == user_inp].index.tolist()
inp = inp[0]
movie['similarity'] = rating_matrix.iloc[inp]
movie.columns=['movie_id', 'title', 'release_date', 'similarity']
movie.head(5)

,movie_id,title,release_date,similarity
0,1,Toy Story (1995),01-Jan-1995,0.238346
1,2,GoldenEye (1995),01-Jan-1995,0.175394
2,3,Four Rooms (1995),01-Jan-1995,0.000000
3,4,Get Shorty (1995),01-Jan-1995,0.222119
4,5,Copycat (1995),01-Jan-1995,0.109961


In [15]:
# 测试集构造
# 此时的data就为用户评分表，即复现中的movie_rating
# 创建用户-电影评分表
rating_test_matrix = test_data.pivot_table(index = ['item_id'],columns = ['user_id'],
                                values = 'rating').reset_index(drop = True)
rating_test_matrix.fillna(0, inplace = True)
user_item_test = rating_test_matrix
user_item_test.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
user_item_test.shape

(1491, 943)

In [42]:
# 测试集构造见物品相似矩阵
movie_similarity_test = 1 - pairwise_distances(rating_test_matrix.values, metric = "cosine")
np.fill_diagonal(movie_similarity_test,0)
rating_matrix_test = pd.DataFrame(movie_similarity_test)
rating_matrix_test.head()

,0,1,2,3,4,5,6,7,8,9,...,1481,1482,1483,1484,1485,1486,1487,1488,1489,1490
0,0.000000,0.165272,0.151241,0.167775,0.089217,0.085727,0.131293,0.101301,0.130802,0.041930,...,0.044423,0.0,0.0,0.111056,0.000000,0.0,0.0,0.0,0.0,0.000000
1,0.165272,0.000000,0.098438,0.110813,0.054219,0.000000,0.085199,0.079759,0.097137,0.076444,...,0.000000,0.0,0.0,0.000000,0.106299,0.0,0.0,0.0,0.0,0.159448
2,0.151241,0.098438,0.000000,0.117297,0.100720,0.023361,0.069584,0.032057,0.143805,0.014690,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
3,0.167775,0.110813,0.117297,0.000000,0.106780,0.021986,0.178067,0.114651,0.155300,0.101392,...,0.000000,0.0,0.0,0.136717,0.000000,0.0,0.0,0.0,0.0,0.102538
4,0.089217,0.054219,0.100720,0.106780,0.000000,0.000000,0.097413,0.016884,0.066355,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000


In [43]:
rating_matrix_test.shape

(1491, 1491)

In [37]:
# 测试集top-N推荐
user_inp = "Four Rooms (1995)" 
inp = movie[movie['title'] == user_inp].index.tolist()
inp = inp[0]
movie['similarity'] = rating_matrix_test.iloc[inp]
movie.columns=['movie_id', 'title', 'release_date', 'similarity']
movie.head(5)

,movie_id,title,release_date,similarity
0,1,Toy Story (1995),01-Jan-1995,0.151241
1,2,GoldenEye (1995),01-Jan-1995,0.098438
2,3,Four Rooms (1995),01-Jan-1995,0.000000
3,4,Get Shorty (1995),01-Jan-1995,0.117297
4,5,Copycat (1995),01-Jan-1995,0.100720


In [44]:
# 训练集预测，求得RMSE
# 将dataframe形式的用户物品相似矩阵转为numpy 数组
rating_matrix_array = rating_matrix.values
user_item_pre = rating_matrix_array.dot(movie_similarity) / np.array([np.abs(movie_similarity).sum(axis = 1)])

pre_flatten = user_item_pre
rating_matrix_flatten = rating_matrix_array
from sklearn.metrics import mean_squared_error
from math import sqrt
error_train = sqrt(mean_squared_error(pre_flatten, rating_matrix_flatten))
print(error_train)

0.06401731646769637


In [47]:
# 测试集预测，求得RMSE
rating_test_matrix_array = rating_matrix_test.values
user_item_test_pre = rating_test_matrix_array.dot(movie_similarity_test) / np.array([np.abs(movie_similarity_test).sum(axis = 1)])

test_pre_flatten = user_item_test_pre
test_rating_matrix_flatten = rating_test_matrix_array
error_test = sqrt(mean_squared_error(test_pre_flatten, test_rating_matrix_flatten))

In [48]:
print(error_test)

0.04753928213271828
